In [19]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from torch.utils.data import (DataLoader)  # Gives easier dataset managment and creates mini batches
#import torchvision  # torch package for vision related things
#import torchvision.datasets as datasets  # Has standard datasets we can import in a nice way
#import torchvision.transforms as transforms
import pandas as pd
import random
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from tqdm.auto import tqdm
from transformers import DistilBertTokenizer

In [20]:
# Run when executing on Google Colab
#from google.colab import drive
#drive.mount('/content/drive')

#!cp "/content/drive/MyDrive/Colab Notebooks/Deep Learning/DeepLProject/SpamDataset.zip" "./SpamDataset.zip"
#!unzip SpamDataset.zip

In [21]:
# Split the downloaded Youtube01-Psy.csv file into train, validation and test sets
data = pd.read_csv('SpamDataset/Youtube05-Combined.csv')
# data = pd.read_csv('TSwift_ShakeItOff_Spam.csv')
data = data.sample(frac=1).reset_index(drop=True)  # Shuffle the data
train_data = data[:int(0.7*len(data))]
val_data = data[int(0.7*len(data)):int(0.85*len(data))]
test_data = data[int(0.85*len(data)):]

# Tokenize the data using distillbert
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Turn pd dataframe into a tokenized PyTorch dataset that DataLoader can use via the SpamDataset class
class SpamDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        comment_tokens = row['CONTENT']
        label = row['CLASS']
        return comment_tokens, label
    
    def collate_fn(self, batch):
        comments = [row[0] for row in batch]
        labels = [row[1] for row in batch]
        tokens = tokenizer.batch_encode_plus(comments, truncation=True, padding='max_length', max_length=30, return_tensors='pt')
        return tokens, torch.tensor(labels)
    
train_dataset = SpamDataset(train_data)
val_dataset = SpamDataset(val_data)
test_dataset = SpamDataset(test_data)


# Load Data and collate it
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=train_dataset.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, collate_fn=val_dataset.collate_fn)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, collate_fn=test_dataset.collate_fn)

In [22]:
# Print a random sample from the train_loader to confirm correct loading and tokenization
sample = next(iter(train_loader))
print(sample)
print([samples.size() for samples in sample[0]['input_ids']])


({'input_ids': tensor([[  101,  2065,  1996,  ...,     0,     0,     0],
        [  101, 16770,  1024,  ...,     0,     0,     0],
        [  101,  7632,   999,  ...,     0,     0,     0],
        ...,
        [  101,  1008,  1008,  ...,  2026,  2047,   102],
        [  101,  4931,   999,  ...,     0,     0,     0],
        [  101,  1045,  2293,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}, tensor([0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
        0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0]))
[torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), torch.Size([30]), tor

# Models
Create all different Models that we are testing. Most have LSTMs in common, but 2 add improvements

In [23]:
# Recurrent neural network with LSTM (many-to-one) for sequence classification to produce a binary output
## The network includes an Embedding layer, an LSTM layer and a Linear layer
class RNN_LSTM(nn.Module):
    def __init__(self, embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(embedding_dict, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x: (batch_size, max_seq_length)
        embedding = self.dropout(self.embedding(x))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output, (h_n, c_n) = self.lstm(embedding)
        # output shape: (batch_size, max_seq_length, hidden_size)
        # h_n shape: (num_layers, batch_size, hidden_size)
        out = self.fc(h_n[-1, :, :])
        # out shape: (batch_size, num_classes)
        return out
    
# Bidirectional LSTM network for sequence classification to produce a binary output
## The network includes an Embedding layer, a Bidirectional LSTM layer and a Linear layer
class BiLSTM(nn.Module):
    def __init__(self, embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(embedding_dict, embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)
        

    def forward(self, x):
        # x: (batch_size, max_seq_length)
        embedding = self.dropout(self.embedding(x))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output, (h_n, c_n) = self.lstm(embedding)
        # output shape: (batch_size, max_seq_length, hidden_size*2)
        # h_n shape: (num_layers*2, batch_size, hidden_size)
        out = self.fc(torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=1))
        # out shape: (batch_size, num_classes)
        return out

# An Attention-LSTM network for sequence classification to produce a binary output
## The network includes an Embedding layer, an Attention layer, an LSTM layer and a Linear layer
class AttentionLSTM(nn.Module):
    def __init__(self, embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout):
        super(AttentionLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.embedding = nn.Embedding(embedding_dict, embedding_size)
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.attention = nn.Linear(hidden_size, 1)
        self.fc = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        # x: (batch_size, max_seq_length)
        embedding = self.dropout(self.embedding(x))
        # embedding shape: (batch_size, max_seq_length, embedding_size)
        output, (h_n, c_n) = self.lstm(embedding)
        # output shape: (batch_size, max_seq_length, hidden_size)
        # h_n shape: (num_layers, batch_size, hidden_size)
        attention_weights = F.softmax(self.attention(output), dim=1)
        # attention_weights shape: (batch_size, max_seq_length, 1)
        out = torch.sum(attention_weights * output, dim=1)
        # out shape: (batch_size, hidden_size)
        out = self.fc(out)
        # out shape: (batch_size, num_classes)
        return out

In [24]:
# Store all the predictions in model_predictions and the actual labels in actual_labels
tokens = []
original_comments = []
model_predictions = []
actual_labels = []

# Validation
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    total_loss = 0.0
    predictions = []
    targets = []
    model.eval()

    with torch.no_grad():
        for data, target in loader:
            data = {key: value.to(device) for key, value in data.items()}
            target = target.to(device)

            scores = model(data['input_ids'])
            _, predicted = scores.max(1)
            # Store decoded original comments, model predictions and actual labels 
            original_comments.extend([tokenizer.decode(data['input_ids'][i]) for i in range(len(data['input_ids']))])
            tokens.extend(data['input_ids'])
            model_predictions.extend(predicted)
            actual_labels.extend(target)
            num_correct += (predicted == target).sum()
            num_samples += predicted.size(0)

            loss = criterion(scores, target)
            total_loss += loss


            predictions.extend(predicted.tolist())
            targets.extend(target.tolist())

    # model.train()
    average_loss = total_loss / num_samples
    accuracy = num_correct/num_samples
    return accuracy, predictions, targets, average_loss

In [25]:
def results(model):
    train_acc, train_preds, train_targets, train_loss = check_accuracy(train_loader, model)
    val_acc, val_preds, val_targets, train_loss = check_accuracy(val_loader, model)
    test_acc, test_preds, test_targets, train_loss = check_accuracy(test_loader, model)

    print(f'Train accuracy: {train_acc}')
    print(f'Validation accuracy: {val_acc}')
    print(f'Test accuracy: {test_acc}')

    train_cm = confusion_matrix(train_targets, train_preds)
    val_cm = confusion_matrix(val_targets, val_preds)
    test_cm = confusion_matrix(test_targets, test_preds)

    print(f'Train Confusion Matrix:\n{train_cm}')
    print(f'Validation Confusion Matrix:\n{val_cm}')
    print(f'Test Confusion Matrix:\n{test_cm}')

    # Classification report
    train_report = classification_report(train_targets, train_preds)
    val_report = classification_report(val_targets, val_preds)
    test_report = classification_report(test_targets, test_preds)

    print(f'Train Classification Report:\n{train_report}')
    print(f'Validation Classification Report:\n{val_report}')
    print(f'Test Classification Report:\n{test_report}')

# Training

In [26]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()

# Set the hyperparameters for all the models     # TODO: analyze these by model
embedding_dict = tokenizer.vocab_size
embedding_size = 128
hidden_size = 256
NUM_LAYERS = [2,4,8,16,32,64]
num_classes = 2
DROPOUT = [0.3, 0.5, 0.7]
LR = [0.001, 0.003, 0.005, 0.007, 0.009]
NUM_EPOCH = [10, 25, 50]
# init LSTM model

for num_layers in NUM_LAYERS:
    for dropout in DROPOUT:
        for learning_rate in LR:
            for num_epochs in NUM_EPOCH:
                model_LSTM = RNN_LSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
                optimizer_LSTM = optim.Adam(model_LSTM.parameters(), lr=learning_rate)
                # RNN_LSTM
                print('RNN_LSTM Training')
                for epoch in range(num_epochs):
                    model_LSTM.train()
                    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
                    for batch_idx, (data, targets) in loop:
                        data = {key: value.to(device) for key, value in data.items()}
                        targets = targets.to(device)
                        # forward
                        scores = model_LSTM(data['input_ids'])
                        loss = criterion(scores, targets)
                        # backward
                        optimizer_LSTM.zero_grad()
                        loss.backward()
                        # gradient descent or adam step
                        optimizer_LSTM.step()
                        # update progress bar
                        loop.set_description(f'Epoch [{epoch}/{num_epochs}]')
                        loop.set_postfix(loss=loss.item())
                # init BiLSTM model
                model_BiLSTM = BiLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
                optimizer_BiLSTM = optim.Adam(model_BiLSTM.parameters(), lr=learning_rate)
                # BiLSTM
                print('BiLSTM Training')
                for epoch in range(num_epochs):
                    model_BiLSTM.train()
                    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
                    for batch_idx, (data, targets) in loop:
                        data = {key: value.to(device) for key, value in data.items()}
                        targets = targets.to(device)
                        # forward
                        scores = model_BiLSTM(data['input_ids'])
                        loss = criterion(scores, targets)
                        # backward
                        optimizer_BiLSTM.zero_grad()
                        loss.backward()
                        # gradient descent or adam step
                        optimizer_BiLSTM.step()
                        # update progress bar
                        loop.set_description(f'Epoch [{epoch}/{num_epochs}]')
                        loop.set_postfix(loss=loss.item())

                # init AttentionLSTM model
                model_AttentionLSTM = AttentionLSTM(embedding_dict, embedding_size, hidden_size, num_layers, num_classes, dropout).to(device)
                optimizer_AttentionLSTM = optim.Adam(model_AttentionLSTM.parameters(), lr=learning_rate)

                # AttentionLSTM
                print('AttentionLSTM Training')
                for epoch in range(num_epochs):
                    model_AttentionLSTM.train()
                    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
                    for batch_idx, (data, targets) in loop:
                        data = {key: value.to(device) for key, value in data.items()}
                        targets = targets.to(device)
                        # forward
                        scores = model_AttentionLSTM(data['input_ids'])
                        loss = criterion(scores, targets)
                        # backward
                        optimizer_AttentionLSTM.zero_grad()
                        loss.backward()
                        # gradient descent or adam step
                        optimizer_AttentionLSTM.step()
                        # update progress bar
                        loop.set_description(f'Epoch [{epoch}/{num_epochs}]')
                        loop.set_postfix(loss=loss.item())
                print("Current Num_Layers: ", num_layers)
                print("Current Dropout: ", dropout)
                print("Current Learning Rate: ", learning_rate)
                print("Current Num_Epochs: ", num_epochs)
                results(model_LSTM)
                results(model_BiLSTM)
                results(model_AttentionLSTM)

RNN_LSTM Training


  0%|          | 0/22 [00:00<?, ?it/s]

BiLSTM Training


AttentionLSTM Training


Current Num_Layers:  2
Current Dropout:  0.3
Current Learning Rate:  0.001
Current Num_Epochs:  10
Train accuracy: 0.9408327341079712
Validation accuracy: 0.9112628102302551
Test accuracy: 0.8809523582458496
Train Confusion Matrix:
[[600  61]
 [ 20 688]]
Validation Confusion Matrix:
[[124  17]
 [  9 143]]
Test Confusion Matrix:
[[121  28]
 [  7 138]]
Train Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.91      0.94       661
           1       0.92      0.97      0.94       708

    accuracy                           0.94      1369
   macro avg       0.94      0.94      0.94      1369
weighted avg       0.94      0.94      0.94      1369

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.88      0.91       141
           1       0.89      0.94      0.92       152

    accuracy                           0.91       293
   macro avg       0.91      0.91      

...

BiLSTM Training


AttentionLSTM Training


Current Num_Layers:  2
Current Dropout:  0.7
Current Learning Rate:  0.005
Current Num_Epochs:  25
Train accuracy: 0.9963477253913879
Validation accuracy: 0.9488054513931274
Test accuracy: 0.9387755393981934
Train Confusion Matrix:
[[658   3]
 [  2 706]]
Validation Confusion Matrix:
[[132   9]
 [  6 146]]
Test Confusion Matrix:
[[138  11]
 [  7 138]]
Train Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       661
           1       1.00      1.00      1.00       708

    accuracy                           1.00      1369
   macro avg       1.00      1.00      1.00      1369
weighted avg       1.00      1.00      1.00      1369

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.94      0.95       141
           1       0.94      0.96      0.95       152

    accuracy                           0.95       293
   macro avg       0.95      0.95      

BiLSTM Training


Epoch [7/50]:  91%|█████████ | 20/22 [00:04<00:00,  5.23it/s, loss=0.0769]

In [ ]:
results(model_LSTM)
results(model_BiLSTM)
results(model_AttentionLSTM)

Train accuracy: 0.9839298725128174
Validation accuracy: 0.9419795274734497
Test accuracy: 0.9455782175064087
Train Confusion Matrix:
[[659  11]
 [ 11 688]]
Validation Confusion Matrix:
[[127   7]
 [ 10 149]]
Test Confusion Matrix:
[[140   7]
 [  9 138]]
Train Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       670
           1       0.98      0.98      0.98       699

    accuracy                           0.98      1369
   macro avg       0.98      0.98      0.98      1369
weighted avg       0.98      0.98      0.98      1369

Validation Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       134
           1       0.96      0.94      0.95       159

    accuracy                           0.94       293
   macro avg       0.94      0.94      0.94       293
weighted avg       0.94      0.94      0.94       293

Test Classification Report:
 

In [ ]:
# Print the original comments, model predictions and actual labels
for i in range(5):
    print(f'Comment: {original_comments[i]}')
    print(f'Tokens: {tokens[i]}')
    print(f'Prediction: {model_predictions[i]}')
    print(f'Actual: {actual_labels[i]}')
    print('')


Comment: [CLS] love your song makes me happy [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Tokens: tensor([ 101, 2293, 2115, 2299, 3084, 2033, 3407,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0])
Prediction: 0
Actual: 0

Comment: [CLS] is that tiger called'katty purry '? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
Tokens: tensor([  101,  2003,  2008,  6816,  2170,  1005, 10645,  3723, 16405, 12244,
         1005,  1029,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])
Prediction: 0
Actual: 0

Comment: [CLS] sign up for free on tsu and start making money on social media add / follow me and ill add follow you!! http : / [SEP]
Tokens: tensor([  